<img src="emir_color_853x315_trans.png" alt="EMIR logo" style="float: right; width: 150px;"/>

# Precomputed calibrations

## Bad-pixel mask

File: [BadPixMask_F.fits](BadPixMask_F.fits)

In order to apply the mask:
```bash
$ numina-bpm filein.fits \
  --bpm BacPixMask_F.fits \
  --outfile fileout.fits
```

For additional help execute:
```bash
$ numina-bpm --help
```


## Multislit boundary models

Multislit boundary models, computed from continuum images with even and odd slitlets alternatively opened and closed, are available for each grism + filter combination:

- grism J + filter J: [refined_multislit_param_grism_J_filter_J.json](refined_multislit_param_grism_J_filter_J.json)
- grism H + filter H: [refined_multislit_param_grism_H_filter_H.json](refined_multislit_param_grism_H_filter_H.json)
- grism K + filter Ksp: [refined_multislit_param_grism_K_filter_Ksp.json](refined_multislit_param_grism_K_filter_Ksp.json)
- grism LR + filter YJ: [refined_multislit_param_grism_LR_filter_YJ.json](refined_multislit_param_grism_LR_filter_YJ.json)
- grism LR + filter HK: [refined_multislit_param_grism_LR_filter_HK.json](refined_multislit_param_grism_LR_filter_HK.json)

The script `pyemir-overplot_boundary_model` can be used to overplot the boundary model over any FITS image. In addition, by using the optional arguments `--ds9reg_boundaries` or/and `--ds9reg_frontiers`, this script can generate ds9 region files with the boundaries and frontiers of all the slitlets.

```bash
$ pyemir-overplot_boundary_model filein.fits \
  --fitted_bound_param refined_multislit_param_grism_J_filter_J.json \
  --ds9reg_boundaries ds9_boundaries.reg \
  --ds9reg_frontiers ds9_frontiers.reg
```

For additional help execute:

```bash
$ pyemir-overplot_boundary_model --help
```

## Flatfield (pixel-to-pixel variations)

Pixel-to-pixel variation flatfield: ideally computed from continuum lamp exposures with lamp ON and OFF.

*Warning: spurious signal (ghosts?) present*

- grism J + filter J: [continuum_ls05_J_J_on-off_flat.fits](continuum_ls05_J_J_on-off_flat.fits)
- grism H + filter H: [continuum_ls06_H_H_on-off_flat.fits](continuum_ls06_H_H_on-off_flat.fits)
- grism K + filter Ksp: [continuum_ls08_K_Ksp_on-off_flat.fits](continuum_ls08_K_Ksp_on-off_flat.fits)
- grism LR + filter YJ: TBD
- grism LR + filter HK: TBD


## Rectification and wavelength calibration transformations

Library of rectification and wavelength calibration transformations (computed from arc exposures):

- grism J + filter J: [rect_wpoly_MOSlibrary_grism_J_filter_J.json](rect_wpoly_MOSlibrary_grism_J_filter_J.json)
- grism H + filter H: [rect_wpoly_MOSlibrary_grism_H_filter_H.json](rect_wpoly_MOSlibrary_grism_H_filter_H.json)
- grism K + filter Ksp: [rect_wpoly_MOSlibrary_grism_K_filter_Ksp.json](rect_wpoly_MOSlibrary_grism_K_filter_Ksp.json)
- grism LR + filter YJ: TBD
- grism LR + filter HK: TBD

The previous files contain, for each grism+filter configuration, the variation of every coefficient corresponding to the rectification and wavelength calibration polynomials, for a collection of CSU configurations (i.e., different `csu_bar_slit_center` values).

The process to proceed with the rectification and wavelength calibration of a particular image is carried out in two steps.

<div class="alert alert-block alert-danger">
Please, note that in its current state, this functionality still needs to be improved in order to:
<ul>
    <li> Carry out the rectfication process using an interpolation technique that takes into account fraction of pixels preserving the flux (at present the code uses the nearest-neighbor algorithm, which preserves the flux but leads to a typical zigzag pattern; on the other side, this approach is fast).</li>
    <li> Take into account DTU shifts (that still need to be properly calibrated).</li>
</ul>
In addition, it is important to realize that the method described below makes use of an interpolation technique based on calibration images obtained in a discrete sampling of the CSU configuration.
<br><br>
<b>For all those reasons, it is not unexpected that the absolute wavelength calibration might suffer from small systematic offsets.</b>
</div>

**Step 1: derive the rectification and wavelength calibration polynomials for a particular CSU configuration**

The script `pyemir-evaluate_rect_wpoly` is employed to obtain a linear interpolation of the rectification and wavelength calibration coefficients from the two nearest `csu_bar_slit_center` configurations.

```bash
$ pyemir-evaluate_rect_wpoly infile.fits \
  --rect_wpoly_MOSlibrary rect_wpoly_MOSlibrary_grism_J_filter_J.json \
  --out_rect_wpoly coef_rect_wpoly_from_mosmodel.json
```

For additional help execute:

```bash
$ pyemir-evaluate_rect_wpoly --help
```

**Step 2: apply the rectfication and wavelength calibration polynomials for a particular CSU configuration**

Next, the generated JSON file `coef_rect_wpoly_from_mosmodel.json` is be employed to rectify and wavelength calibrate any image obtained with the same instrumental setup and CSU configuration.

```bash
$ pyemir-apply_rect_wpoly infile.fits \
  --coef_rect_wpoly coef_rect_wpoly_from_mosmodel.json \
  --outfile infile_rw.fits \
  --outfile_rectified_only infile_r.fits
```

Note that the last parameter, `--outfile_rectified_only` is optional.

For additional help execute:

```bash
$ pyemir-apply_rect_wpoly --help
```

In [1]:
%%bash
date

Thu Apr  5 15:33:57 CEST 2018
